In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls
%cd 'drive/My Drive/trainingandtestdata'
!ls 

drive  sample_data
/content/drive/My Drive/trainingandtestdata
clean_tweet.csv  testdata.csv  training.csv


In [0]:
import pandas as pd
path = 'training.csv'
train = pd.read_csv(path, encoding='latin-1')

In [0]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
import os 
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

%matplotlib inline
pd.set_option('display.max_colwidth', 100)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
print(train.info())
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
0                                                                                                                      1599999 non-null int64
1467810369                                                                                                             1599999 non-null int64
Mon Apr 06 22:19:45 PDT 2009                                                                                           1599999 non-null object
NO_QUERY                                                                                                               1599999 non-null object
_TheSpecialOne_                                                                                                        1599999 non-null object
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    1599999 non-null object
dtypes: int64(2), object(4)
memo

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School to...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you a..."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [0]:
cols = train.columns
train.loc[len(train)]=[0, cols[1], cols[2], cols[3], cols[4], cols[5]] 

In [0]:
train.columns = ['sentiment','id','date','query_string','user','text']

In [0]:
train.info()
train.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
sentiment       1600000 non-null int64
id              1600000 non-null object
date            1600000 non-null object
query_string    1600000 non-null object
user            1600000 non-null object
text            1600000 non-null object
dtypes: int64(1), object(5)
memory usage: 85.4+ MB


,sentiment,id,date,query_string,user,text
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interviews! â« http://blip.fm/~8bmta
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me for details
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur
1599998,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H
1599999,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Thi..."


In [0]:
train.sentiment.value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [0]:
train.drop(['id','date','query_string','user'],axis = 1,inplace=True)

Now the data cleaning phase will start, it will consist of - 


*   @ removal
*   HTML stripping
*   #removal
*   removing links
*   UTF-8 boom 

We will look at each procedure properly 







In [0]:
# '@' mention removal using regex

def remove_user_mention(text):
  pattern = r'@[A-Za-z0-9_]+'
  stripped = re.sub(pattern, '', text)
  return stripped


# below is just an example to show how '@' mentions are rmoeved
#--------------------------------------------------------------------
text = train.text[174]
print(text)
print(remove_user_mention(text))
#--------------------------------------------------------------------

@machineplay I'm so sorry you're having to go through this. Again.  #therapyfail
 I'm so sorry you're having to go through this. Again.  #therapyfail


In [0]:
# removing HTML encoding
def decode_HTML(text):
  decoded = BeautifulSoup(text, 'lxml')
  return decoded.get_text()
  
# below is just an example to show how HTML is decoded
#--------------------------------------------------------------------
text = train.text[278]
print(text)
print(decode_HTML(text))
#--------------------------------------------------------------------

Whinging. My client&amp;boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&amp;reviewed correctly. 
Whinging. My client&boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&reviewed correctly. 


In [0]:
# removing URL links
def remove_URL(text):
  pat1 = 'https?://[^ ]+'
  pat2 = 'www.[^ ]+'
  combined_pat = r'|'.join((pat1, pat2))
  stripped = re.sub(combined_pat, '', text)
  return stripped
  
# below is just an example to show how URL links are removed
#--------------------------------------------------------------------
text = train.text[1599999]
print(text)
print(remove_URL(text))
#--------------------------------------------------------------------

@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
@switchfoot  - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D


In [0]:
# removing UTF - Byte Order Mark
def remove_BOM(text):
  stripped = text.replace(u"ï¿½", " ")
  return stripped


# below is just an example to show how URL links are removed
#--------------------------------------------------------------------
text = train.text[225]
print(text)
print(remove_BOM(text))
#--------------------------------------------------------------------

Tuesdayï¿½ll start with reflection ï¿½n then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers 
Tuesday ll start with reflection  n then a lecture in Stress reducing techniques. That sure might become very useful for us accompaniers 


In [0]:
# checking for apostrophes existing in sentences

apos = "'"
has_apos = train[train['text'].str.contains(apos)]
print(has_apos.head(10))

    sentiment                                                                                                 text
0           0  is upset that he can't update his Facebook by texting it... and might cry as a result  School to...
3           0  @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you a...
6           0  @LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?
7           0                                                                 @Tatiana_K nope they didn't have it 
9           0                                                          spring break in plain city... it's snowing 
11          0       @caregiving I couldn't bear to watch it.  And I thought the UA loss was embarrassing . . . . .
13          0  @smarrison i would've been the first, but i didn't have a gun.    not really though, zac snyder'...
15          0        Hollis' death scene will hurt me severely to watch on film 

### `The above output suggests that there are 489504 entries which contain apostrophes. 
Without handling these cases differently the function "remove_hashtag" as mentioned below will remove apostrophes and the result will be like this:

**Intput**: 

"*is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!*"


**Ouput**:  

"*is upset that he can t update his facebook by texting it and might cry as a result school today also blah*"

You can see that the word "can't" is replaced by "can t" which changes the whole meaning of the word. We will handle this differently.

In [0]:
# rmeoving apostrophes from text
def remove_apostrophe(phrase):
  phrase = re.sub(r"won\'t", "will not", phrase)
  phrase = re.sub(r"can\'t", "can not", phrase)

    # general
  phrase = re.sub(r"n\'t", " not", phrase)
  phrase = re.sub(r"\'re", " are", phrase)
  phrase = re.sub(r"\'s", " is", phrase)
  phrase = re.sub(r"\'d", " would", phrase)
  phrase = re.sub(r"\'ll", " will", phrase)
  phrase = re.sub(r"\'t", " not", phrase)
  phrase = re.sub(r"\'ve", " have", phrase)
  phrase = re.sub(r"\'m", " am", phrase)
  return phrase

text = has_apos['text'].iloc[0]
print(text)
apos_cleared = remove_apostrophe(text)
print(apos_cleared)
  

I'm sweating my forthcoming trip to E3.  If I can't find someone to crash with while I'm out there, I may be screwed.  
I am sweating my forthcoming trip to E3.  If I can not find someone to crash with while I am out there, I may be screwed.  


In [0]:
# Removal of '#' using regex

# Remember this process should be done in the end after you have modified all the data because it removes any other character other than alphabet.

def remove_hashtag(text):
  pattern = r'[^a-zA-Z]'
  stripped = re.sub(pattern, ' ', text)
  return stripped


# below is just an example to show how hashtags are removed
#--------------------------------------------------------------------
text = apos_cleared
print(text)
print(remove_hashtag(text))
#--------------------------------------------------------------------


I am sweating my forthcoming trip to E3.  If I can not find someone to crash with while I am out there, I may be screwed.  
I am sweating my forthcoming trip to E    If I can not find someone to crash with while I am out there  I may be screwed   


In [0]:
# Removal of stopwords

def remove_stop(text):
  tokens = tok.tokenize(text)
  stop_words = stopwords.words('english')
  new_tokens = [word for word in tokens if word not in stop_words]
#  print(new_tokens)
  return (" ".join(new_tokens)).strip()


# below is just an example to show how hashtags are removed
#--------------------------------------------------------------------
text = apos_cleared
print(text)
print(remove_stop(text))
#--------------------------------------------------------------------


I am sweating my forthcoming trip to E3.  If I can not find someone to crash with while I am out there, I may be screwed.  
I sweating forthcoming trip E3 . If I find someone crash I , I may screwed .


In [0]:
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

def clean_tweet(text):
  HTML_decoded = decode_HTML(text)
  user_mention_removed = remove_user_mention(HTML_decoded)
  url_removed = remove_URL(user_mention_removed)
  apos_removed = remove_apostrophe(url_removed)
  hashtag_removed = remove_hashtag(apos_removed)
  BOM_removed = remove_BOM(hashtag_removed)
  lower_case = BOM_removed.lower()
  stop_removed = remove_stop(lower_case)
  words = tok.tokenize(stop_removed)
  return (" ".join(words)).strip()
  

In [0]:
clean_tweet_texts = []
clean_tweet_texts.clear()
for i in range(0,100):
  clean_tweet_texts.append(clean_tweet(train['text'][i]))

In [0]:
clean_tweet_texts 

['upset update facebook texting might cry result school today also blah',
 'dived many times ball managed save rest go bounds',
 'whole body feels itchy like fire',
 'behaving mad see',
 'whole crew',
 'need hug',
 'hey long time see yes rains bit bit lol fine thanks',
 'nope',
 'que muera',
 'spring break plain city snowing',
 'pierced ears',
 'could bear watch thought ua loss embarrassing',
 'counts idk either never talk anymore',
 'would first gun really though zac snyder doucheclown',
 'wish got watch miss premiere',
 'hollis death scene hurt severely watch film wry directors cut',
 'file taxes',
 'ahh ive always wanted see rent love soundtrack',
 'oh dear drinking forgotten table drinks',
 'day get much done',
 'one friend called asked meet mid valley today time sigh',
 'baked cake ated',
 'week going hoped',
 'blagh class tomorrow',
 'hate call wake people',
 'going cry sleep watching marley',
 'im sad miss lilly',
 'ooooh lol leslie ok leslie get mad',
 'meh almost lover excepti

In [0]:
print("Cleaning and parsing the tweets...\n")

clean_tweet_texts.clear()
for i in range(0,1600000):
    if( (i+1)%50000 == 0 ):
        print("Tweets %d has been processed" % ( i+1 ))                                                                    
    clean_tweet_texts.append(clean_tweet(train['text'][i]))

Cleaning and parsing the tweets...

Tweets 50000 has been processed
Tweets 100000 has been processed
Tweets 150000 has been processed
Tweets 200000 has been processed
Tweets 250000 has been processed
Tweets 300000 has been processed
Tweets 350000 has been processed
Tweets 400000 has been processed
Tweets 450000 has been processed
Tweets 500000 has been processed
Tweets 550000 has been processed
Tweets 600000 has been processed
Tweets 650000 has been processed
Tweets 700000 has been processed
Tweets 750000 has been processed
Tweets 800000 has been processed
Tweets 850000 has been processed
Tweets 900000 has been processed
Tweets 950000 has been processed
Tweets 1000000 has been processed
Tweets 1050000 has been processed
Tweets 1100000 has been processed
Tweets 1150000 has been processed
Tweets 1200000 has been processed
Tweets 1250000 has been processed
Tweets 1300000 has been processed
Tweets 1350000 has been processed
Tweets 1400000 has been processed
Tweets 1450000 has been processe

In [0]:
clean_tweet_texts[:100]

In [0]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['target'] = train.sentiment
clean_df.head()


,text,target
0,upset update facebook texting might cry result school today also blah,0
1,dived many times ball managed save rest go bounds,0
2,whole body feels itchy like fire,0
3,behaving mad see,0
4,whole crew,0


In [0]:
clean_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
text      1600000 non-null object
target    1600000 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [0]:
clean_df.to_csv('clean_tweet.csv',encoding='utf-8')
csv = 'clean_tweet.csv'
cleaned = pd.read_csv(csv,index_col=0)
cleaned.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,text,target
0,upset update facebook texting might cry result school today also blah,0
1,dived many times ball managed save rest go bounds,0
2,whole body feels itchy like fire,0
3,behaving mad see,0
4,whole crew,0


In [0]:
cleaned.dropna(inplace=True)
cleaned.reset_index(drop=True,inplace=True)
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591427 entries, 0 to 1591426
Data columns (total 2 columns):
text      1591427 non-null object
target    1591427 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.3+ MB
